# Introduction

In this notebook, we demonstrate the steps needed to create an IoT Edge deployable module from the regression model created in the [turbofan regression](./01-turbofan_regression.ipynb) notebook. The steps we will follow are:
   1. Reload experiment and model from the Azure Machine Learning service workspace
   1. Create a scoring script
   1. Create an environment YAML file
   1. Create a container image using the model, scoring script and YAML file
   1. Deploy the container image as a web service 
   1. Test the web service to make sure the container works as expected
   1. Delete the web service
   
><font color=gray>Note: this notebook depends on the workspace, experiment and model created in the [turbofan regression](./01-turbofan_regression.ipynb) notebook.</font>

# Set up notebook

Please ensure that you are running this notebook under the Python 3.6 Kernel. The current kernel is show on the top of the notebook at the far right side of the file menu. If you are not running Python 3.6 you can change it in the file menu by clicking **Kernel->Change Kernel->Python 3.6**

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

## Configure workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **aml_config/.azureml/config.json** and loads the details into an object named `ws`, which is used throughout the rest of the code in this notebook.

In [2]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.model import Model
from azureml.train.automl.run import AutoMLRun

ws = Workspace.from_config(path='./aml_config')

## Load run, experiment and model

Use the model information that we persisted in the [turbofan regression](./01-turbofan_regression.ipynb) noebook to load our model.

In [3]:
import json 

#name project folder and experiment
model_data = json.load(open('./aml_config/model_config.json'))

run_id = model_data['regressionRunId']
experiment_name = model_data['experimentName']
model_id = model_data['modelId']

experiment = Experiment(ws, experiment_name)
automl_run = AutoMLRun(experiment = experiment, run_id = run_id)
model = Model(ws, model_id)

# Create scoring script

The scoring script is the piece of code that runs inside the container and interacts with the model to return a prediction to the caller of web service or Azure IoT Edge module that is running the container. The scoring script is authored knowing the shape of the message that will be sent to the container. In our case, we have chosen to format the message as:

```json
[{
    "DeviceId": 81,
    "CycleTime": 140,
    "OperationalSetting1": 0.0,
    "OperationalSetting2": -0.0002,
    "OperationalSetting3": 100.0,
    "Sensor1": 518.67,
    "Sensor2": 642.43,
    "Sensor3": 1596.02,
    "Sensor4": 1404.4,
    "Sensor5": 14.62,
    "Sensor6": 21.6,
    "Sensor7": 559.76,
    "Sensor8": 2388.19,
    "Sensor9": 9082.16,
    "Sensor10": 1.31,
    "Sensor11": 47.6,
    "Sensor12": 527.82,
    "Sensor13": 2388.17,
    "Sensor14": 8155.92,
    "Sensor15": 8.3214,
    "Sensor16": 0.03,
    "Sensor17": 393.0,
    "Sensor18": 2388.0,
    "Sensor19": 100.0,
    "Sensor20": 39.41,
    "Sensor21": 23.5488
}]
```

In [4]:
script_file_name = 'score.py'

In [5]:
%%writefile $script_file_name
import pickle
import json
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path(model_name = '<<modelname>>')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
def unpack_message(raw_data):
    message_data = json.loads(raw_data)
    # convert single message to list 
    if type(message_data) is dict:
        message_data = [message_data]
    return message_data
    
def extract_features(message_data):
    X_data = []
    sensor_names = ['Sensor'+str(i) for i in range(1,22)]
    
    for message in message_data:
        # select sensor data from the message dictionary
        feature_dict = {k: message[k] for k in (sensor_names)}
        X_data.append(feature_dict)
    
    X_df = pd.DataFrame(X_data)
    return np.array(X_df[sensor_names].values)

def append_predict_data(message_data, y_hat):
    message_df = pd.DataFrame(message_data)
    message_df['PredictedRul'] = y_hat
    return message_df.to_dict('records')

def log_for_debug(log_message, log_data):
    print("*****%s:" % log_message)
    print(log_data)
    print("******")

def run(raw_data):
    log_for_debug("raw_data", raw_data)
    
    message_data = unpack_message(raw_data)
    log_for_debug("message_data", message_data)
    
    X_data = extract_features(message_data)
    log_for_debug("X_data", X_data)
   
    # make prediction
    y_hat = model.predict(X_data)
    
    response_data = append_predict_data(message_data, y_hat)
    return response_data

Overwriting score.py


### Update the scoring script with the actual model ID

In [6]:
# Substitute the actual model id in the script file.

with open(script_file_name, 'r') as cefr:
    content = cefr.read()

with open(script_file_name, 'w') as cefw:
    cefw.write(content.replace('<<modelname>>', model.name))

1649

## Create YAML file for the environment

The YAML file provides the information about the dependencies for the model we will deploy. 

### Get azureml versions

First we will use the run to retrieve the version of the azureml packages used to train the model. 

>Warnings about the version of the SDK not matching with the training version are expected 

In [30]:
best_run, fitted_model = automl_run.get_output()
iteration = int(best_run.get_properties()['iteration'])
dependencies = automl_run.get_run_sdk_dependencies(iteration = iteration)


In [22]:
dependencies

{'azureml-dataprep-native': '38.0.0',
 'azureml-dataprep': '4.5.7',
 'azureml-dataprep-rslex': '2.11.4',
 'azureml-automl-core': '1.46.1',
 'azureml-training-tabular': '1.46.1',
 'azureml-automl-runtime': '1.46.1',
 'azureml-train-automl-runtime': '1.46.1',
 'azureml-defaults': '1.46.0',
 'azureml-pipeline-core': '1.46.0',
 'azureml-train-automl-client': '1.46.0',
 'azureml-train-restclients-hyperdrive': '1.46.0',
 'azureml-responsibleai': '1.46.0',
 'azureml-dataset-runtime': '1.46.0',
 'azureml-mlflow': '1.46.0',
 'azureml-train-core': '1.46.0',
 'azureml-telemetry': '1.46.0',
 'azureml-interpret': '1.46.0',
 'azureml-core': '1.46.0',
 'azureml-inference-server-http': '0.7.6'}

In [9]:
for p in ['azureml-train-automl-runtime', 'azureml-core']:
    print('{}\t{}'.format(p, dependencies[p]))

azureml-train-automl-runtime	1.46.1
azureml-core	1.46.0


### Write YAML file 

Write the initial YAML file to disk and update the dependencies for azureml to match with the training versions. This is not strictly needed in this notebook because the model likely has been generated using the current SDK version. However, we include this for completeness for the case when an experiment was trained using a previous SDK version.

In [10]:
import azureml.core
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','pandas','psutil'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

# Substitute the actual version number in the environment file.
with open(conda_env_file_name, 'r') as cefr:
    content = cefr.read()



'myenv.yml'

In [42]:
from azureml.core import Environment
myenv = Environment(name="myenv-inference")

conda_dep = CondaDependencies()

conda_dep.add_pip_package("azureml-sdk[automl]~=1.46.0")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("psutil")

myenv.python.conda_dependencies=conda_dep



In [43]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(
    entry_script=script_file_name,
    environment=myenv
)

In [44]:
from azureml.core.webservice import AciWebservice

service_name = 'my-custom-env-service'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config,
                          overwrite=True)


In [45]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-30 06:48:56+00:00 Creating Container Registry if not exists.
2022-11-30 06:48:56+00:00 Registering the environment.
2022-11-30 06:48:58+00:00 Building image..
2022-11-30 06:59:26+00:00 Generating deployment configuration.
2022-11-30 06:59:27+00:00 Submitting deployment to compute..
2022-11-30 06:59:34+00:00 Checking the status of deployment my-custom-env-service..
2022-11-30 07:02:18+00:00 Checking the status of inference endpoint my-custom-env-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [11]:
# with open(conda_env_file_name, 'w') as cefw:
#    cefw.write(content.replace(azureml.core.VERSION, dependencies['azureml-core']))

558

## Create a container image

Use the scoring script and the YAML file to create a container image in the workspace. The image will take several minutes to create.

## Deploy image as a web service on Azure Container Instance

Deploy the image we just created as web service on Azure Container Instance (ACI). We will use this web service to test that our model/container performs as expected. 

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
aci_service_name = 'edge-ml-rul-01'


aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "digits", 'type': "automl_RUL"}, 
                                               description = 'test service for Edge ML RUL')

print ("Deploying service: %s" % aci_service_name)

aci_service = Webservice.deploy_from_image(deployment_config = aci_config,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)

aci_service.wait_for_deployment(True)
print ("Service state: %s" % aci_service.state)

## Load test data

To save a couple of steps at this point, we serialized the test data that we loaded in the [turbofan regression](./01-turbofan_regression.ipynb) notebook. Here we deserialize that data to use it to test the web service.

In [46]:
import pandas as pd
from sklearn.externals import joblib
import numpy

test_df = pd.read_csv("data/WebServiceTest.csv")

test_df.head(5)

,Unnamed: 0,Unit,CycleTime,OperationalSetting1,OperationalSetting2,OperationalSetting3,Sensor1,Sensor2,Sensor3,Sensor4,...,Sensor16,Sensor17,Sensor18,Sensor19,Sensor20,Sensor21,MaxCycle,TestEndRUL,RUL,RulClass
0,0,1,1,-0.00,-0.00,100.00,518.67,641.94,1581.93,1396.93,...,0.03,391,2388,100.00,39.07,23.45,233,44,276,Full
1,1,1,2,0.00,-0.00,100.00,518.67,642.02,1584.86,1398.90,...,0.03,391,2388,100.00,39.04,23.48,233,44,275,Full
2,2,1,3,0.00,-0.00,100.00,518.67,641.68,1581.78,1391.92,...,0.03,391,2388,100.00,39.10,23.42,233,44,274,Full
3,3,1,4,0.00,0.00,100.00,518.67,642.20,1584.53,1395.34,...,0.03,392,2388,100.00,38.97,23.48,233,44,273,Full
4,4,1,5,-0.00,0.00,100.00,518.67,642.46,1589.03,1395.86,...,0.03,391,2388,100.00,39.09,23.39,233,44,272,Full


## Predict one message at a time

Once the container/model is deployed to and Azure IoT Edge device it will receive messages one at a time. Send a few messages in that mode to make sure everything is working.

In [49]:
my_aci_service = AciWebservice(ws,service_name)

In [53]:
import json
import pandas as pd

# reformat data as list of messages
X_message = test_df.head(5).to_dict('record')
for row in X_message:
    row_data = json.dumps(row)
    print(row_data)

{"Unnamed: 0": 0, "Unit": 1, "CycleTime": 1, "OperationalSetting1": -0.0017, "OperationalSetting2": -0.0004, "OperationalSetting3": 100.0, "Sensor1": 518.67, "Sensor2": 641.94, "Sensor3": 1581.93, "Sensor4": 1396.93, "Sensor5": 14.62, "Sensor6": 21.58, "Sensor7": 554.56, "Sensor8": 2387.93, "Sensor9": 9048.65, "Sensor10": 1.3, "Sensor11": 47.09, "Sensor12": 521.89, "Sensor13": 2387.94, "Sensor14": 8133.48, "Sensor15": 8.376, "Sensor16": 0.03, "Sensor17": 391, "Sensor18": 2388, "Sensor19": 100.0, "Sensor20": 39.07, "Sensor21": 23.4468, "MaxCycle": 233, "TestEndRUL": 44, "RUL": 276, "RulClass": "Full"}
{"Unnamed: 0": 1, "Unit": 1, "CycleTime": 2, "OperationalSetting1": 0.0006, "OperationalSetting2": -0.0002, "OperationalSetting3": 100.0, "Sensor1": 518.67, "Sensor2": 642.02, "Sensor3": 1584.86, "Sensor4": 1398.9, "Sensor5": 14.62, "Sensor6": 21.58, "Sensor7": 554.1, "Sensor8": 2387.94, "Sensor9": 9046.53, "Sensor10": 1.3, "Sensor11": 47.08, "Sensor12": 521.85, "Sensor13": 2388.01, "Senso

In [50]:
import json
import pandas as pd

# reformat data as list of messages
X_message = test_df.head(5).to_dict('record')

result_list = []
for row in X_message:
    row_data = json.dumps(row)
    row_result = my_aci_service.run(input_data=row_data)
    result_list.append(row_result[0])

result_df = pd.DataFrame(result_list)
residuals = result_df['RUL'] - result_df['PredictedRul']
result_df['Residual'] = residuals
result_df[['CycleTime','RUL','PredictedRul','Residual']]


ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '481', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 30 Nov 2022 07:12:44 GMT', 'Server': 'nginx/1.18.0 (Ubuntu)', 'X-Ms-Client-Request-Id': '17699a29-7811-4025-8571-795bc5b6f8c9', 'X-Ms-Request-Id': '17699a29-7811-4025-8571-795bc5b6f8c9', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.7', 'X-Request-Id': '1f66f53b-5ae3-4214-957f-734b84c27812'}
Content: b'DataException:\n\tMessage: Expected column(s) 0 not found in fitted data.\n\tInnerException: None\n\tErrorResponse \n{\n    "error": {\n        "code": "UserError",\n        "message": "Expected column(s) 0 not found in fitted data.",\n        "target": "X",\n        "inner_error": {\n            "code": "BadArgument",\n            "inner_error": {\n                "code": 

WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '481', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 30 Nov 2022 07:12:44 GMT', 'Server': 'nginx/1.18.0 (Ubuntu)', 'X-Ms-Client-Request-Id': '17699a29-7811-4025-8571-795bc5b6f8c9', 'X-Ms-Request-Id': '17699a29-7811-4025-8571-795bc5b6f8c9', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.7', 'X-Request-Id': '1f66f53b-5ae3-4214-957f-734b84c27812'}
Content: b'DataException:\n\tMessage: Expected column(s) 0 not found in fitted data.\n\tInnerException: None\n\tErrorResponse \n{\n    "error": {\n        "code": "UserError",\n        "message": "Expected column(s) 0 not found in fitted data.",\n        "target": "X",\n        "inner_error": {\n            "code": "BadArgument",\n            "inner_error": {\n      

## Predict entire set

To make sure the model as a whole is working as expected, we send the test set in bulk to the model, save the predictions, and calculate the residual.

In [52]:
import json
import pandas as pd

X_messages = test_df.to_dict('record')
raw_data = json.dumps(X_messages)

result_list = my_aci_service.run(input_data=raw_data)
result_df = pd.DataFrame(result_list)
residuals = result_df['RUL'] - result_df['PredictedRul']
result_df['Residual'] = residuals

y_test = result_df['RUL']
y_pred = result_df['PredictedRul']

ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '166', 'Content-Type': 'text/html', 'Date': 'Wed, 30 Nov 2022 07:21:14 GMT', 'Server': 'nginx/1.18.0 (Ubuntu)', 'X-Ms-Request-Id': '2d4c428a-349f-45e4-b540-979fd5b9b438'}
Content: b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.18.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '166', 'Content-Type': 'text/html', 'Date': 'Wed, 30 Nov 2022 07:21:14 GMT', 'Server': 'nginx/1.18.0 (Ubuntu)', 'X-Ms-Request-Id': '2d4c428a-349f-45e4-b540-979fd5b9b438'}
Content: b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.18.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '166', 'Content-Type': 'text/html', 'Date': 'Wed, 30 Nov 2022 07:21:14 GMT', 'Server': 'nginx/1.18.0 (Ubuntu)', 'X-Ms-Reques

## Plot actuals vs. predicted

To validate the shape of the model, plot the actual RUL against the predicted RUL for each cycle and device.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

fig, ax = plt.subplots()
fig.set_size_inches(8, 4)

font_size = 12

g = sns.regplot(y='PredictedRul', x='RUL', data=result_df, fit_reg=False, ax=ax)
lim_set = g.set(ylim=(0, 500), xlim=(0, 500))
plot = g.axes.plot([0, 500], [0, 500], c=".3", ls="--");

rmse = ax.text(16,450,'RMSE = {0:.2f}'.format(numpy.sqrt(mean_squared_error(y_test, y_pred))), fontsize = font_size)
r2 = ax.text(16,425,'R2 Score = {0:.2f}'.format(r2_score(y_test, y_pred)), fontsize = font_size)

xlabel = ax.set_xlabel('Actual RUL', size=font_size)
ylabel = ax.set_ylabel('Predicted RUL', size=font_size)


## Delete web service

Now that we are confident that our container and model are working well, delete the web service.

In [ ]:
from azureml.core.webservice import Webservice
aci_service = Webservice(ws, 'edge-ml-rul-01')

aci_service.delete()